In [1]:
# Importing libraries
import pandas as pd
import emoji
import re
import string

import json

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

p_list = ['facebook', 'tensorflow', 'microsoft', 'bitcoin','opencv',
          'ansible','flutter','kubernetes','roslyn','typescript', 'dartlang','integrate']
project_name = 'opencv'
Clean_Method = "CM1"
# checkpoint = 225
# in : 2475
checkpoint = 225
# checkpoint = 750

train_data = pd.read_csv('../'+Clean_Method+'/data/csvdata/'+project_name+'_train.csv')
test_data = pd.read_csv('../'+Clean_Method+'/data/csvdata/'+project_name+'_test.csv')

print('train data : ../'+Clean_Method+'/data/csvdata/'+project_name+'_train.csv')
print('test data : ../'+Clean_Method+'/data/csvdata/'+project_name+'_test.csv')

학습 데이터 : ../CM1/data/csvdata/opencv_train.csv
테스트 데이터 : ../CM1/data/csvdata/opencv_test.csv


In [2]:
print(train_data.shape)
print(test_data.shape)

(300, 6)
(300, 6)


In [3]:
import os
os.environ['HF_TOKEN']="API_KEY"

In [4]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [6]:

print(f"pytorch version {torch.__version__}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")


pytorch version 2.2.2
working on cuda:0


In [7]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [8]:


   
def generate_test_prompt(data_point):
      return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|> GitHub Issue Report Classifier <|eot_id|><|start_header_id|>user<|end_header_id|> Classify, IN ONLY 1 WORD, the following GitHub issue as 'feature', 'bug', or 'question' based on its title and body:"+str(data_point["content"])+"<|eot_id|><|start_header_id|>assistant<|end_header_id|>".strip()


X_test = pd.DataFrame(test_data.apply(generate_test_prompt, axis=1), columns=["text"])

y_true = test_data.label


In [9]:
def evaluate(y_true, y_pred):
    labels =  ["bug", "feature", "question"]
    mapping = {'question': 2, 'feature': 1, 'bug': 0}
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print(project_name)
    print('\nClassification Report:')
    
    # file_name = "../CM1/result/metric/epoch3/"+project_name+"_"+str(checkpoint)+"_cm.txt"
    file_name = "../CM1/result/metric/"+project_name+"_"+str(checkpoint)+"_cm.txt"
    with open(file_name, "w") as text_file:
        print(classification_report(y_true, y_pred, digits=4), file=text_file)
    
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')

    print(conf_matrix)

In [10]:
# model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
model_name="../CM1/model/epoch3/"+project_name+"-llama-classifier/checkpoint-"+str(checkpoint)   
token_name="../CM1/model/epoch3/"+project_name+"-llama-classifier/checkpoint-"+str(checkpoint)     



print("model 이름: "+model_name)
print("토크나이저 이름: "+token_name)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 1024 #2048
tokenizer = AutoTokenizer.from_pretrained(token_name, max_seq_length=max_seq_length)
tokenizer.pad_token_id = tokenizer.eos_token_id

model 이름: ../CM1/model/epoch3/opencv-llama-classifier/checkpoint-225
토크나이저 이름: ../CM1/model/epoch3/opencv-llama-classifier/checkpoint-225


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
def predict(X_test, model, tokenizer):
    y_pred = []
    y_answer = []
    for i in tqdm(range(len(X_test))):
        try:
            prompt = X_test.iloc[i]["text"]
            pipe = pipeline(task="text-generation", 
                            model=model, 
                            tokenizer=tokenizer, 
                            max_new_tokens = 1, 
                            temperature = 0.1,
                            # 여기
                        )
            result = pipe(prompt)
            answer = result[0]['generated_text'].split("<|end_header_id|>")[-1]
            y_answer.append(answer)
            if "question" in answer:
                y_pred.append("question")
            elif "feature" in answer:
                y_pred.append("feature")
            elif "bug" in answer:
                y_pred.append("bug")
            else: 
                y_pred.append("empty")
                # print(result)
        except Exception as e:
            # print(len(tokenizer(X_test.iloc[i]["text"])['input_ids']))
            y_pred.append("empty")
            y_answer.append('empty')
            # print(e)
            # print('bug')
  
    return y_pred, y_answer

In [12]:
from sklearn.metrics import (accuracy_score, 
                             recall_score, 
                             precision_score, 
                             f1_score)

from transformers import EarlyStoppingCallback, IntervalStrategy

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [13]:
y_pred, y_answer= predict(X_test, model, tokenizer)
result=evaluate(y_true, y_pred)
print(project_name)
print(result)

100%|██████████| 300/300 [01:12<00:00,  4.12it/s]

Accuracy: 0.770
Accuracy for label 0: 0.450
Accuracy for label 1: 0.900
Accuracy for label 2: 0.960
opencv

Classification Report:

Confusion Matrix:
[[45 27 28]
 [ 0 90 10]
 [ 3  1 96]]
opencv
None


In [14]:
evaluation = pd.DataFrame({'text': X_test["text"], 
                           'y_true':y_true, 
                           'y_pred': y_pred, 
                            'y_answer': y_answer},
                         )
evaluation.to_csv("../CM1/result/metric/predict/"+project_name+"_"+str(checkpoint)+"_result.csv", index=False)

In [15]:
print(evaluation)

                                                  text   y_true    y_pred  \
0    <|begin_of_text|><|start_header_id|>system<|en...      bug  question   
1    <|begin_of_text|><|start_header_id|>system<|en...      bug  question   
2    <|begin_of_text|><|start_header_id|>system<|en...      bug       bug   
3    <|begin_of_text|><|start_header_id|>system<|en...      bug  question   
4    <|begin_of_text|><|start_header_id|>system<|en...      bug  question   
..                                                 ...      ...       ...   
295  <|begin_of_text|><|start_header_id|>system<|en...  feature   feature   
296  <|begin_of_text|><|start_header_id|>system<|en...  feature   feature   
297  <|begin_of_text|><|start_header_id|>system<|en...  feature   feature   
298  <|begin_of_text|><|start_header_id|>system<|en...  feature   feature   
299  <|begin_of_text|><|start_header_id|>system<|en...  feature   feature   

     y_answer  
0    question  
1    question  
2         bug  
3    questi

In [16]:
!pip install playsound